# **Previsão de preço de carros.**
<!--
<img src="https://miro.medium.com/v2/resize:fit:988/0*tA5OjppLK627FfFo" alt="image" width="250" height="auto">
-->
Nesse projeto eu vou construir um modelo de **Machine Learning** que faça a previsão  do preço de venda de um veículo.

Irei utilizar a Metodologia **CRISP-DM** para me basear nos passos a serem seguinos no projeto.

1. **Compreensão do Negócio (Business Understanding/Problem)** 

1. **Compreensão dos Dados (Data Understanding)** 
1. **Preparação dos Dados (Data Preparation)** 
1. **Modelagem (Modeling)** 
1. **Avaliação (Evaluation)**
1. **Implantação (Deployment)** 

# **1. Problema de Negóscio**

*   Uma pessoa precisa revender seu veículo porém ela **não sabe** qual valor deve cobrar no mesmo.
*   1.1 Qual é o contexto?
    *   ...
*   1.2 Quais são os objetivos do projeto?
    *   Identificar as características que influenciam os preços de venda dos veículos.
    *   Criar um modelo capaz de prever o valor de venda de um novo veículo.
    *   ...
*   1.3 Quais são os benefícios?
    *   Vender o veículo com o valor *ótimo* de acordo com **suas características**.
    *   **Reduzir as perdas de dinheiro** na revenda do veículo.
    *   **Enconomizar esforços de energia/tempo** em tentar adivinhar o valor ideal.

Como resultado, o **problema de negócio** será resolvido.

### **Importando as bibliotecas necessárias.**

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
# import matplotlib as mpl
import seaborn as sns

# Display options.
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.width', 1000)

# **2. Compreendendo os dados**
*   Os dados foram obtidos através de web scraping do site  [Olx](https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?) e **são de propiedade de olx.com**
*   O repositório do projeto de web scraping completo se encontra [aqui](https://github.com/PatrickLeal/olx-veiculos-web-scraping).
*   Os dados contém informações sobre os veículos anunciados no site. 

In [2]:
file_path = glob.glob('../data_lake/silver/*.csv')
df_carros = pd.read_csv(file_path[0], parse_dates=['DATA_PUBLICACAO'])           

In [3]:
df_carros.sample(3)

,TITULO,PRECO_BRL,DESCRICAO,OPCIONAIS,CEP,IMAGEM,PERFIL_CARRO,DATA_PUBLICACAO,TEMPO_PUBLICACAO,COD_PUBLICACAO,TIPO_ANUNCIO,MODELO,PORTAS,CATEGORIA,CAMBIO,COR,POTENCIA_DO_MOTOR,QUILOMETRAGEM,TIPO_DE_DIREÇÃO,COMBUSTIVEL,FINAL_DE_PLACA,MARCA,ANO,TIPO_DE_VEICULO,POSSUI_KIT_GNV
11825,Renault Logan 2023 1.0 12v sce flex zen manual,62490.00,RENAULT LOGAN 1.0 12V SCE FLEX ZEN MANUAL - 20...,"['Air bag', 'Som', 'Vidro elétrico']",45023325,https://img.olx.com.br/images/50/5004839246716...,https://ba.olx.com.br/regiao-de-vitoria-da-con...,2024-09-13,10:55,1337419207,profissional,RENAULT LOGAN ZEN FLEX 1.0 12V 4P MEC.,4 portas,"Carros, vans e utilitários",Manual,Branco,1.0,48622,Hidráulica,Flex,5.00,RENAULT,2023,Sedã,Não
13639,TOYOTA HILUX SW4 SRV D4-D 4x4 3.0 TDI Dies. Aut,131900.00,A procura de um Toyota Hilux SW4 SRV D4-D 4x4 ...,"['Alarme', 'Air bag', 'Ar condicionado', 'Câme...",14403050,https://img.olx.com.br/images/83/8354846706102...,https://sp.olx.com.br/regiao-de-ribeirao-preto...,2024-09-17,11:31,1331866108,profissional,TOYOTA HILUX SW4 SRV D4-D 4X4 3.0 TDI DIES. AUT,4 portas,"Carros, vans e utilitários",Automático,Branco,NaN,206930,Hidráulica,Diesel,1.00,TOYOTA,2012,Pick-up,Não
5452,Chevrolet Onix 2023 1.0 flex manual,69890.00,CHEVROLET ONIX 1.0 FLEX MANUAL - 2023 / 2023 ...,"['Alarme', 'Ar condicionado', 'Trava elétrica']",79006001,https://img.olx.com.br/images/17/1784606727726...,https://ms.olx.com.br/mato-grosso-do-sul/autos...,2024-09-10,11:22,1323885153,profissional,CHEVROLET HATCH 1.0 12V FLEX MEC. 4P,4 portas,"Carros, vans e utilitários",Manual,Branco,1.0,63905,Elétrica,Flex,1.00,CHEVROLET,2023,Hatch,Não


### **Dicionário de dados**

|VARIÁVEL| DESCRIÇÃO | TIPO |
|--------|-----------|------|
|**TITULO:**| Título do anúncio| string |
|**PRECO_BRL:**| Preço em Reais R$| int |
|**DESCRICAO:**| Descrição do anúncio deixada pelo anunciante| string |
|**OPCIONAIS:**| São *features* dos veículos informadas pelo anunciante| string |
|**CEP:**| Localização veículo deixada pelo anunciante | int |
|**IMAGEM:**| Link da foto de perfil do anúncio | string |
|**PERFIL_CARRO:**| Link para o anúncio original | string |
|**DATA_PUBLICACAO:**| Date em que o anúncio foi criado| date |
|**TEMPO_PUBLICACAO:**| Hora em que o anúncio foi criado| time |
|**COD_PUBLICACAO:**| Código do anúncio| int |
|**TIPO_ANUNCIO:**| Tipo de anúncio| string|
|**ANO:**| Informa o Ano doveículo| int |
|**MARCA:**| Nome da marca do veículo| string |
|**CAMBIO:**| Tipo de câmbio do veículo | string |
|**POSSUI_KIT_GNV:**| Se o veículo possui kit GNV| bool |
|**PORTAS:**| Quantidade de portas que o veículo possui| int |
|**POTENCIA_DO_MOTOR:**| Informa a potência do motor | string |
|**TIPO_DE_VEICULO:**| Informa o tipo de veículo |string |
|**FINAL_DE_PLACA:**| Informa o final da placa| int |
|**COR:**| Informa a cor do veículo| string |
|**QUILOMETRAGEM:**| Informa a quilometragem do veículo| int |
|**COMBUSTIVEL:**| Informa o o tipo de combustível usado pelo veículo| string|
|**TIPO_DE_DIREÇÃO:**| Informa o tipo de direção do veículo| string |
|**MODELO:**| Informa o modelo do veículo|string |
|**CATEGORIA:**| Informa e qual categoria o veículo foi anunciado |string |

`PRECO_BRL`  será nossa variável ***target***

### **Informações gerais dos dados:**

In [4]:
df_carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18805 entries, 0 to 18804
Data columns (total 25 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TITULO             object        
 1   PRECO_BRL          float64       
 2   DESCRICAO          object        
 3   OPCIONAIS          object        
 4   CEP                int64         
 5   IMAGEM             object        
 6   PERFIL_CARRO       object        
 7   DATA_PUBLICACAO    datetime64[ns]
 8   TEMPO_PUBLICACAO   object        
 9   COD_PUBLICACAO     int64         
 10  TIPO_ANUNCIO       object        
 11  MODELO             object        
 12  PORTAS             object        
 13  CATEGORIA          object        
 14  CAMBIO             object        
 15  COR                object        
 16  POTENCIA_DO_MOTOR  object        
 17  QUILOMETRAGEM      int64         
 18  TIPO_DE_DIREÇÃO    object        
 19  COMBUSTIVEL        object        
 20  FINAL_DE_PLACA     float64  

In [5]:
print(f"O dataset possui {df_carros.shape[0]} linhas e {df_carros.shape[1]} colunas.")

O dataset possui 18805 linhas e 25 colunas.


**Visualisando algumas estatísticas:**

In [6]:
df_carros.describe().T

,count,mean,min,25%,50%,75%,max,std
PRECO_BRL,18644.00,88322.99,0.00,52990.00,71990.00,105900.00,4950000.00,75600.07
CEP,18805.00,39824387.67,1013010.00,12040001.00,31160190.00,68040050.00,99680000.00,29547174.27
DATA_PUBLICACAO,18805,2024-09-12 12:47:30.901356032,2024-07-12 00:00:00,2024-09-09 00:00:00,2024-09-13 00:00:00,2024-09-17 00:00:00,2024-09-17 00:00:00,NaN
COD_PUBLICACAO,18805.00,1329885212.11,614262154.00,1329351445.00,1334684764.00,1336423149.00,1338383745.00,18651014.29
QUILOMETRAGEM,18805.00,72242.72,0.00,43502.00,53219.00,92094.00,9999999.00,107297.16
FINAL_DE_PLACA,15763.00,4.40,0.00,2.00,4.00,7.00,9.00,2.87


**Análise:**
-   o dataset possui anúncios com o **preço** de venda **igual a 0**, o que não faz sentido nesse contexto
-   alguns dados estão com **tipos errados** e precisam ser tradados

**Verificando valores nulos e duplicados:**

In [7]:
print("Porcentagem de valores nulos.")
(df_carros.isnull().sum() / df_carros.shape[0]) * 100

Porcentagem de valores nulos.


TITULO               0.00
PRECO_BRL            0.86
DESCRICAO            0.00
OPCIONAIS            0.00
CEP                  0.00
IMAGEM               0.00
PERFIL_CARRO         0.00
DATA_PUBLICACAO      0.00
TEMPO_PUBLICACAO     0.00
COD_PUBLICACAO       0.00
TIPO_ANUNCIO        24.53
MODELO               0.00
PORTAS               1.75
CATEGORIA            0.00
CAMBIO               0.44
COR                  1.80
POTENCIA_DO_MOTOR    4.53
QUILOMETRAGEM        0.00
TIPO_DE_DIREÇÃO      6.47
COMBUSTIVEL          0.46
FINAL_DE_PLACA      16.18
MARCA                0.00
ANO                  0.00
TIPO_DE_VEICULO     11.79
POSSUI_KIT_GNV       0.00
dtype: float64

In [8]:
print(f"Quantidade de valores duplicados: {df_carros.duplicated().sum()}")

Quantidade de valores duplicados: 2789


**Ações a serem tomada:**
*   remover duplicatas
*   remover colunas que tenham mais de 15% de valores nulos
*   remover coluna *CATEGORIA* porque ela só possui 1  valor único
*   alterar os tipos de dados das colunas:
    1. ANO
    1. PORTAS
    1. POTENCIA_DO_MOTOR
    1. OPCIONAIS
*   remover valores nulos

In [9]:
import os
import sys

module_path = os.path.abspath(os.getcwd() + '//..')
if module_path not in sys.path:
    sys.path.append(module_path)

from scripts import cleaning_tool

df_cleaned = cleaning_tool.limpar_dados_silver(df_carros)

In [10]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12477 entries, 0 to 12476
Data columns (total 23 columns):
 #   Column             Dtype         
---  ------             -----         
 0   titulo             object        
 1   preco_brl          float64       
 2   descricao          object        
 3   opcionais          object        
 4   cep                int64         
 5   imagem             object        
 6   perfil_carro       object        
 7   data_publicacao    datetime64[ns]
 8   tempo_publicacao   object        
 9   cod_publicacao     int64         
 10  modelo             object        
 11  portas             int64         
 12  cambio             object        
 13  cor                object        
 14  potencia_do_motor  category      
 15  quilometragem      int64         
 16  tipo_de_direção    object        
 17  combustivel        object        
 18  marca              object        
 19  ano                int64         
 20  tipo_de_veiculo    object   

In [11]:
# df_carros.drop_duplicates(inplace=True)
print("Porcentagem de valores nulos.")
(df_cleaned.isnull().sum() / df_carros.shape[0]) * 100

Porcentagem de valores nulos.


titulo              0.00
preco_brl           0.00
descricao           0.00
opcionais           0.00
cep                 0.00
imagem              0.00
perfil_carro        0.00
data_publicacao     0.00
tempo_publicacao    0.00
cod_publicacao      0.00
modelo              0.00
portas              0.00
cambio              0.00
cor                 0.00
potencia_do_motor   0.00
quilometragem       0.00
tipo_de_direção     0.00
combustivel         0.00
marca               0.00
ano                 0.00
tipo_de_veiculo     0.00
possui_kit_gnv      0.00
qtd_opcionais       0.00
dtype: float64

Agora não há mais dados nulos e seus tipos estão corretos.

# **3. Dividir os dados em treino e teste**
*   A **análise exploratória de dados** será feita nos dados de treino na intenção de evitar *data leakage*
*   O objetivo principal da divisão é verificar o quão bem o modelo pode fazer previsões em **dados que ele nunca viu antes**. 
*   Se o modelo for avaliado apenas nos **dados de treino**, o resultado pode ser tendencioso, pois o modelo já viu aqueles dados.